In [17]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np
import matplotlib.pyplot as plt #Package for visualization
import re #importing package for Regular expression operations
from sklearn.model_selection import train_test_split #Package for splitting the data
from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical
from keras.preprocessing.text import Tokenizer #Tokenization
from keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils import to_categorical

In [2]:
import pandas as pd

# Load the dataset as a Pandas DataFrame
dataset = pd.read_csv('Sentiment.csv')

# Select only the necessary columns 'text' and 'sentiment' without generating SettingWithCopyWarning
data = dataset[['text', 'sentiment']].copy()

# Apply lowercase transformation to the 'text' column
data.loc[:, 'text'] = data['text'].str.lower()

# Apply regex substitution to remove non-alphanumeric characters from the 'text' column
data.loc[:, 'text'] = data['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))


In [3]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [4]:
for idx, row in data.iterrows():
    data.iloc[idx, 0] = row.iloc[0].replace('rt', ' ') #removing retweets

In [5]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix


In [6]:
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons

In [7]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
# print(model.summary())

In [8]:
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test data split

In [30]:
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 13s - loss: 0.8294 - accuracy: 0.6493 - 13s/epoch - 44ms/step
144/144 - 2s - loss: 0.7536 - accuracy: 0.6649 - 2s/epoch - 14ms/step
0.7535722255706787
0.6649191975593567


In [10]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


#1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [29]:
model.save('sentimentAnalysis.h5') #saving the model 

In [28]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [13]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [14]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")


1/1 - 0s - 274ms/epoch - 274ms/step
[0.7652533  0.17330918 0.06143747]
Neutral


#2. Apply GridSearchCV on the source code provided in the class

In [ ]:
pip install scikeras

In [16]:
from scikeras.wrappers import KerasClassifier #importing Keras classifier

from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(model=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

744/744 - 30s - loss: 0.8294 - accuracy: 0.6474 - 30s/epoch - 40ms/step
186/186 - 1s - 1s/epoch - 7ms/step
744/744 - 27s - loss: 0.8272 - accuracy: 0.6496 - 27s/epoch - 37ms/step
186/186 - 1s - 1s/epoch - 6ms/step
744/744 - 28s - loss: 0.8290 - accuracy: 0.6412 - 28s/epoch - 37ms/step
186/186 - 1s - 1s/epoch - 6ms/step
744/744 - 28s - loss: 0.8295 - accuracy: 0.6436 - 28s/epoch - 38ms/step
186/186 - 2s - 2s/epoch - 9ms/step
744/744 - 30s - loss: 0.8195 - accuracy: 0.6430 - 30s/epoch - 40ms/step
186/186 - 2s - 2s/epoch - 10ms/step
Epoch 1/2
744/744 - 30s - loss: 0.8314 - accuracy: 0.6449 - 30s/epoch - 40ms/step
Epoch 2/2
744/744 - 27s - loss: 0.6853 - accuracy: 0.7094 - 27s/epoch - 37ms/step
186/186 - 2s - 2s/epoch - 10ms/step
Epoch 1/2
744/744 - 30s - loss: 0.8232 - accuracy: 0.6513 - 30s/epoch - 40ms/step
Epoch 2/2
744/744 - 29s - loss: 0.6813 - accuracy: 0.7133 - 29s/epoch - 39ms/step
186/186 - 1s - 1s/epoch - 7ms/step
Epoch 1/2
744/744 - 25s - loss: 0.8287 - accuracy: 0.6412 - 25s/e